In [1]:
!pip install transformers==4.25.1
!pip install datasets==2.8.0
!pip install rouge
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 24.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wh

In [2]:
import random
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import os
import json
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback,
    BartConfig,
    BartForConditionalGeneration,
    AutoModelForMaskedLM


)
from datasets import Dataset
from tqdm import tqdm
import torch


In [3]:
def data_mining(data):
  body_list = []
  text_list = []
  total_list = []
  with open(data) as f:
        data = json.load(f)
  for datum in data["data"]:
      body_list.append(datum['body'])  
  for text in body_list:
    text_list.append(text)
  for utt in text_list:
      utt_list = []
      for i in range(len(utt)):
          utt_list.append(utt[i]['utterance'])
      total_list.append(utt_list)
  return total_list

def data_load(filename):
    text = []

    for file in tqdm(filename):
      total_list = data_mining(file)
      for data in total_list:
        text.append("[sep]".join(data))

    return text


In [4]:
# def data_mining2(data):
#   body_list = []
#   text_list = []
#   total_list = []
#   with open(data) as f:
#         data = json.load(f)
#   for datum in data["data"]:
#       body_list.append(datum['body'])  
#   for text in body_list:
#     text_list.append(text)
#   for utt in text_list:
#       utt_list = []
#       for i in range(len(utt)):
#           utt_list.append(utt[i]['utterance'])
#       total_list.append(utt_list)
#   return total_list[:582]

# def data_load2(filename):
#     text = []

#     for file in tqdm(filename):
#       total_list = data_mining2(file)
#       for data in total_list:
#         text.append("[sep]".join(data))

#     return text

In [5]:
dirname = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_train"
filenames = os.listdir(dirname) 
train_full_filename = []

for filename in filenames:
    fn = os.path.join(dirname, filename)
    if dirname + '/.ipynb_checkpoints' != fn:
        train_full_filename.append(fn)

dirname2 = "/content/drive/MyDrive/인공지능/생성요약프로젝트/data/[라벨]한국어SNS_valid"
filenames2 = os.listdir(dirname2) 
val_full_filename = []

for filename in filenames2:
    fn2 = os.path.join(dirname2, filename)
    if dirname + '/.ipynb_checkpoints' != fn2:
        val_full_filename.append(fn2)

In [6]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-\[\]\(\)]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [7]:
train_dataset = data_load(train_full_filename)
train_list = data_process(train_dataset)

val_dataset = data_load(val_full_filename)
val_list = data_process(val_dataset)


100%|██████████| 200004/200004 [00:05<00:00, 36506.33it/s]


In [8]:
train_df = pd.DataFrame(zip(train_list), columns=['Text'])
val_df = pd.DataFrame(zip(val_list), columns=['Text'])

In [9]:
train_df.head()

,Text
0,두번의[sep]싣로[sep]8일만에 사망..[sep]시도했 [sep]헙[sep]헉[...
1,코로나...때문이 아닐까 추측...[sep]대눈물.....[sep]하나 정해야 하니...
2,뭐햇???[sep]오늘 나 시험 10신데 9시 35분에 일어나서 택시타고 옴[sep...
3,와 보령 2.5단계로 올린대[sep]미쳤어 다른지역은 다 내리는 마당에 여기는 올렸...
4,내가 해먹지뭐[sep]#@이름#가 해주는거 먹으랴면 10년은 기다려야하자냐[sep]...


In [10]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)

print(train_data)
print(val_data)

Dataset({
    features: ['Text'],
    num_rows: 1599992
})
Dataset({
    features: ['Text'],
    num_rows: 200004
})


In [11]:
model_checkpoints = "gogamza/kobart-base-v2"


tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoints)

special_words = [
                "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
                "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
                "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
                "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",
                "#개인 및 관계#", "#미용과 건강#", "#상거래(쇼핑)#", "#시사/교육#", "#식음료#", 
                "#여가 생활#", "#일과 직업#", "#주거와 생활#", "#행사#","[sep]"
                ]

tokenizer.add_special_tokens({"additional_special_tokens": special_words})
model.resize_token_embeddings(len(tokenizer))

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Embedding(30032, 768)

In [ ]:
def get_quartiles(lst):
  q1 = np.percentile(lst, 25)
  q2 = np.percentile(lst, 50)
  q3 = np.percentile(lst, 75)
  return q1, q2, q3

In [ ]:
#t_len = [len(tokenizer.encode(s)) for s in tqdm(train_df['Text'])]
#q1, q2, q3 = get_quartiles(t_len)
#print("Q1:", q1)
#print("Q2:", q2)
#print("Q3:", q3)
#print("mean",sum(t_len) / len(t_len))

In [12]:
max_input = 128
max_target = 128
ignore_index = -100# tokenizer.pad_token_id
masking_rate: float = 0.15

def add_ignored_data(inputs, max_len, ignore_index):
  if len(inputs) < max_len:
      pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
      inputs = np.concatenate([inputs, pad])
  else:
      inputs = inputs[:max_len]

  return inputs

def add_padding_data(inputs, max_len, is_masking=False):
    pad_index = tokenizer.pad_token_id
    if is_masking:
        masked_index = torch.rand(len(inputs)) < masking_rate
        for i, m in enumerate(masked_index):
          if m:
            inputs[i] = tokenizer.mask_token_id
    if len(inputs) < max_len:
        pad = [pad_index] * (max_len - len(inputs))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs 


def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []
    bos = tokenizer('<s>')['input_ids']
    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input, is_masking=True))
    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
        label_id[i].append(tokenizer.eos_token_id)   
        dec_input_id = bos
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Text'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])

  0%|          | 0/1600 [00:00<?, ?ba/s]

  0%|          | 0/201 [00:00<?, ?ba/s]

In [15]:
rouge = Rouge()
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)
    
    return rouge.get_scores(pred_str, label_str, avg=True)   

In [16]:
model.config.max_length = 128 
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 5

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=128,  # demo
    per_device_eval_batch_size=256,
    learning_rate=3e-05,
    weight_decay=0.1,
    #label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/logs2",
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch',
)

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?

trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1599992
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 12500
  Number of trainable parameters = 123884544
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.410600,0.368527


***** Running Evaluation *****
  Num examples = 200004
  Batch size = 256
Saving model checkpoint to /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500
Configuration saved in /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500/config.json
Model weights saved in /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/인공지ᄂ

TrainOutput(global_step=12500, training_loss=0.41056640625, metrics={'train_runtime': 11621.6236, 'train_samples_per_second': 137.674, 'train_steps_per_second': 1.076, 'total_flos': 1.2194668191154176e+17, 'train_loss': 0.41056640625, 'epoch': 1.0})